In [1]:
"""
sanity checking the reconstruction loss dataset
"""

'\nsanity checking the reconstruction loss dataset\n'

In [33]:
import json
import os
import numpy as np


In [20]:
# folder containing your JSON files

folder = "/BS/data_mani_compress/work/thesis/thesis/data/datasets/reconstruction_loss_imgnet_train"

# list all files that match the naming pattern
files = [f for f in os.listdir(folder) if f.startswith("reconstruction_errors_") and f.endswith(".json")]

# sort files to ensure correct order
files.sort()
print((files), "files found.")


combined = []

for fname in files:
    fpath = os.path.join(folder, fname)
    with open(fpath, "r") as f:
        data = json.load(f)
        combined.extend(data)  # merge dictionaries

# write combined json
out_path = os.path.join(folder, "reconstruction_errors_all.json")
with open(out_path, "w") as f:
    json.dump(combined, f, indent=2)

print(f"Combined {len(files)} files into {out_path}, total entries = {len(combined)}")

['reconstruction_errors_0000_0070.json', 'reconstruction_errors_0070_0140.json', 'reconstruction_errors_0140_0210.json', 'reconstruction_errors_0210_0280.json', 'reconstruction_errors_0280_0350.json', 'reconstruction_errors_0350_0420.json', 'reconstruction_errors_0420_0490.json', 'reconstruction_errors_0490_0700.json', 'reconstruction_errors_0700_0900.json', 'reconstruction_errors_0900_1100.json', 'reconstruction_errors_10300_10800.json', 'reconstruction_errors_10800_11300.json', 'reconstruction_errors_1100_1400.json', 'reconstruction_errors_1400_1600.json', 'reconstruction_errors_1600_1900.json', 'reconstruction_errors_1900_2200.json', 'reconstruction_errors_2200_2500.json', 'reconstruction_errors_2500_2800.json', 'reconstruction_errors_2800_3200.json', 'reconstruction_errors_3200_3500.json', 'reconstruction_errors_3500_3800.json', 'reconstruction_errors_3800_4300.json', 'reconstruction_errors_4300_4800.json', 'reconstruction_errors_4800_5300.json', 'reconstruction_errors_5300_5800.js

In [21]:
with open("/BS/data_mani_compress/work/thesis/thesis/data/datasets/reconstruction_loss_imgnet_train/reconstruction_errors_all.json", "r") as f:
    combined = json.load(f)

print(f"Total entries in combined file: {len(combined)}")

Total entries in combined file: 3405440


In [28]:
print(combined[-1])

{'image_id': 329599, 'k_value': 256, 'mse_error': 0.11420442909002304, 'vgg_error': 2.1269850730895996}


In [29]:
# get all the vgg errors
vgg_errors = [entry["vgg_error"] for entry in combined]
print(len(vgg_errors), "vgg errors found.")

3405440 vgg errors found.


In [34]:


vgg_errors = np.array(vgg_errors)
print(np.mean(vgg_errors), "mean vgg error")
print(np.min(vgg_errors), "min vgg error")
print(np.max(vgg_errors), "max vgg error")

2.83240806948364 mean vgg error
0.14857468008995056 min vgg error
6.7774505615234375 max vgg error
